In [1]:
from snowflake.snowpark import Session

In [2]:
import logging
import sys

In [3]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

In [30]:
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"kd10688.ap-northeast-2.aws",
        "USER":"snowpark_user",
        "PASSWORD":"Test@12$4",
        "ROLE":"SYSADMIN",
        "DATABASE":"SNOWFLAKE_SAMPLE_DATA",
        "SCHEMA":"TPCH_SF1",
        "WAREHOUSE":"SNOWPARK_ETL_WH"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create() 

def main():
    session = get_snowpark_session()

    context_df = session.sql("select current_role(), current_database(), current_schema(), current_warehouse()")
    context_df.show(2)

    customer_df = session.sql("select c_custkey,c_name,c_phone,c_mktsegment from snowflake_sample_data.tpch_sf1.customer limit 10")
    customer_df.show(5)
if __name__ == '__main__':
    main()  

01:33:51 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
01:33:51 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
01:33:52 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905090937,
"os.name" : Windows

01:33:52 - INFO - query: [select current_role(), current_database(), current_schema(), current_warehouse()...]
01:33:52 - INFO - query execution done
01:33:52 - INFO - Number of results in first chunk: 0
01:33:52 - INFO - query: [SELECT  *  FROM (select current_role(), current_database(), current_schema(), cu...]
01:33:52 - INFO - query execution done
01:33:52 - INFO - Number of results in first chunk: 1
--------------

In [6]:
def traverse_directory(directory,file_extension) -> list:
    local_file_path = []
    file_name = []  # List to store CSV file paths
    partition_dir = []
    print(directory)
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(file_extension):
                file_path = os.path.join(root, file)
                file_name.append(file)
                
                
                partition_dir.append(root.replace(directory, "").replace("\\", "/"))
                local_file_path.append(file_path.replace("\\", "/"))
                print(local_file_path)
                         
     
        
    return file_name, partition_dir, local_file_path

In [7]:
import os
from snowflake.snowpark import Session
import sys
import logging

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
        "ACCOUNT":"kd10688.ap-northeast-2.aws",
        "USER":"snowpark_user",
        "PASSWORD":"Test@12$4",
        "ROLE":"SYSADMIN",
        "DATABASE":"SNOWFLAKE_SAMPLE_DATA",
        "SCHEMA":"TPCH_SF1"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   

def main():
    # Specify the directory path to traverse
    directory_path = '/tmp/snowpark-e2e/'
    csv_file_name, csv_partition_dir , csv_local_file_path= traverse_directory(directory_path,'.csv')
    parquet_file_name, parquet_partition_dir , parquet_local_file_path= traverse_directory(directory_path,'.parquet')
    json_file_name, json_partition_dir , json_local_file_path= traverse_directory(directory_path,'.json')
    stage_location = '@sales_dwh.source.my_internal_stg'

    
    csv_index = 0
    for file_element in csv_file_name:
        put_result = ( 
                    get_snowpark_session().file.put( 
                        csv_local_file_path[csv_index], 
                        stage_location+"/"+csv_partition_dir[csv_index], 
                        auto_compress=False, overwrite=True, parallel=10)
                    )
        #put_result(file_element," => ",put_result[0].status)
        csv_index+=1

    parquet_index = 0
    for file_element in parquet_file_name:

        put_result = ( 
                    get_snowpark_session().file.put( 
                        parquet_local_file_path[parquet_index], 
                        stage_location+"/"+parquet_partition_dir[parquet_index], 
                        auto_compress=False, overwrite=True, parallel=10)
                    )
        #put_result(file_element," => ",put_result[0].status)
        parquet_index+=1
    
    json_index = 0
    for file_element in parquet_file_name:

        put_result = ( 
                    get_snowpark_session().file.put( 
                        json_local_file_path[json_index], 
                        stage_location+"/"+json_partition_dir[json_index], 
                        auto_compress=False, overwrite=True, parallel=10)
                    )
        #put_result(file_element," => ",put_result[0].status)
        json_index+=1  
    
    
if __name__ == '__main__':
    main()

/tmp/snowpark-e2e/
/tmp/snowpark-e2e/
/tmp/snowpark-e2e/


In [8]:
directory_path = 'C:/sales'
traverse_directory(directory_path,'.csv')        

C:/sales
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv']
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv', 'C:/sales/source=IN/format=csv/date=2020-01-02/order-20200102.csv']
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv', 'C:/sales/source=IN/format=csv/date=2020-01-02/order-20200102.csv', 'C:/sales/source=IN/format=csv/date=2020-01-03/order-20200103.csv']
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv', 'C:/sales/source=IN/format=csv/date=2020-01-02/order-20200102.csv', 'C:/sales/source=IN/format=csv/date=2020-01-03/order-20200103.csv', 'C:/sales/source=IN/format=csv/date=2020-01-04/order-20200104.csv']
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv', 'C:/sales/source=IN/format=csv/date=2020-01-02/order-20200102.csv', 'C:/sales/source=IN/format=csv/date=2020-01-03/order-20200103.csv', 'C:/sales/source=IN/format=csv/date=2020-01-04/order-20200104.csv', 'C:/sales/source=IN/format=csv/dat

(['order-20200101.csv',
  'order-20200102.csv',
  'order-20200103.csv',
  'order-20200104.csv',
  'order-20200105.csv',
  'order-20200106.csv',
  'order-20200107.csv',
  'order-20200108.csv',
  'order-20200109.csv',
  'order-20200110.csv',
  'order-20200111.csv',
  'order-20200112.csv',
  'order-20200113.csv',
  'order-20200114.csv',
  'order-20200115.csv',
  'order-20200116.csv',
  'order-20200117.csv',
  'order-20200118.csv',
  'order-20200119.csv',
  'order-20200120.csv',
  'order-20200121.csv',
  'order-20200122.csv',
  'order-20200123.csv',
  'order-20200124.csv',
  'order-20200125.csv',
  'order-20200126.csv',
  'order-20200127.csv',
  'order-20200128.csv',
  'order-20200129.csv',
  'order-20200130.csv',
  'order-20200131.csv'],
 ['/source=IN/format=csv/date=2020-01-01',
  '/source=IN/format=csv/date=2020-01-02',
  '/source=IN/format=csv/date=2020-01-03',
  '/source=IN/format=csv/date=2020-01-04',
  '/source=IN/format=csv/date=2020-01-05',
  '/source=IN/format=csv/date=2020-01-06

In [53]:
## 실행전에 accountamdin으로 db를 만들어서 sysamdin에게 db usage 권한이랑 write권한을 주고 올렸다
## csv  파일 찾아서 

C:/sales
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv']
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv', 'C:/sales/source=IN/format=csv/date=2020-01-02/order-20200102.csv']
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv', 'C:/sales/source=IN/format=csv/date=2020-01-02/order-20200102.csv', 'C:/sales/source=IN/format=csv/date=2020-01-03/order-20200103.csv']
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv', 'C:/sales/source=IN/format=csv/date=2020-01-02/order-20200102.csv', 'C:/sales/source=IN/format=csv/date=2020-01-03/order-20200103.csv', 'C:/sales/source=IN/format=csv/date=2020-01-04/order-20200104.csv']
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv', 'C:/sales/source=IN/format=csv/date=2020-01-02/order-20200102.csv', 'C:/sales/source=IN/format=csv/date=2020-01-03/order-20200103.csv', 'C:/sales/source=IN/format=csv/date=2020-01-04/order-20200104.csv', 'C:/sales/source=IN/format=csv/dat

'C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv'

In [11]:
def main():
    directory_path = 'C:/sales'
    csv_file_name, csv_partition_dir, csv_local_file_path = traverse_directory(directory_path,'.csv')
    
    stage_location = "@SALES_DWH.SOURCE.MY_INTERNAL_STG"
    
    csv_index = 0
    for file_element in csv_file_name:
        stage_path = stage_location +"/"+ csv_partition_dir[csv_index]
    
        put_result = ( 
                    get_snowpark_session().file.put( 
                        csv_local_file_path[csv_index], 
                        stage_location+"/"+csv_partition_dir[csv_index],
                        auto_compress=  False, overwrite=True, parallel=10)
                    )
        csv_index+=1
if __name__ == '__main__':
    main()

C:/sales
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv']
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv', 'C:/sales/source=IN/format=csv/date=2020-01-02/order-20200102.csv']
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv', 'C:/sales/source=IN/format=csv/date=2020-01-02/order-20200102.csv', 'C:/sales/source=IN/format=csv/date=2020-01-03/order-20200103.csv']
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv', 'C:/sales/source=IN/format=csv/date=2020-01-02/order-20200102.csv', 'C:/sales/source=IN/format=csv/date=2020-01-03/order-20200103.csv', 'C:/sales/source=IN/format=csv/date=2020-01-04/order-20200104.csv']
['C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv', 'C:/sales/source=IN/format=csv/date=2020-01-02/order-20200102.csv', 'C:/sales/source=IN/format=csv/date=2020-01-03/order-20200103.csv', 'C:/sales/source=IN/format=csv/date=2020-01-04/order-20200104.csv', 'C:/sales/source=IN/format=csv/dat

01:28:22 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
01:28:23 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905090865,
"os.name" : Windows

01:28:23 - INFO - query: [PUT 'file://C:/sales/source=IN/format=csv/date=2020-01-01/order-20200101.csv' '@...]
01:28:23 - INFO - query execution done
01:28:23 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
01:28:23 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
01:28:24 - INFO

01:28:35 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905082837,
"os.name" : Windows

01:28:35 - INFO - query: [PUT 'file://C:/sales/source=IN/format=csv/date=2020-01-12/order-20200112.csv' '@...]
01:28:35 - INFO - query execution done
01:28:35 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
01:28:35 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
01:28:36 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905082841,
"os.name" : Windows

01:28:36 - INFO - query: [PUT 'file://C:/sales/source=IN/format=csv/date=202

01:28:47 - INFO - query: [PUT 'file://C:/sales/source=IN/format=csv/date=2020-01-23/order-20200123.csv' '@...]
01:28:47 - INFO - query execution done
01:28:47 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
01:28:47 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
01:28:48 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905090913,
"os.name" : Windows

01:28:48 - INFO - query: [PUT 'file://C:/sales/source=IN/format=csv/date=2020-01-24/order-20200124.csv' '@...]
01:28:48 - INFO - query execution done
01:28:48 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-S

In [ ]:
## json 파일 찾아서 

In [58]:
def main():
    directory_path = 'C:/sales'
    csv_file_name, csv_partition_dir, csv_local_file_path = traverse_directory(directory_path,'.json')
    
    stage_location = "@SALES_DWH.SOURCE.MY_INTERNAL_STG"
    
    csv_index = 0
    for file_element in csv_file_name:
        stage_path = stage_location +"/"+ csv_partition_dir[csv_index]
    
        put_result = ( 
                    get_snowpark_session().file.put( 
                        csv_local_file_path[csv_index], 
                        stage_location+"/"+csv_partition_dir[csv_index],
                        auto_compress=  False, overwrite=True, parallel=10)
                    )
        csv_index+=1
if __name__ == '__main__':
    main()

C:/sales
['C:/sales/source=FR/format=json/date=2020-01-01/order-20200102.json']
['C:/sales/source=FR/format=json/date=2020-01-01/order-20200102.json', 'C:/sales/source=FR/format=json/date=2020-01-02/order-20200102.json']
['C:/sales/source=FR/format=json/date=2020-01-01/order-20200102.json', 'C:/sales/source=FR/format=json/date=2020-01-02/order-20200102.json', 'C:/sales/source=FR/format=json/date=2020-01-03/order-20200103.json']
['C:/sales/source=FR/format=json/date=2020-01-01/order-20200102.json', 'C:/sales/source=FR/format=json/date=2020-01-02/order-20200102.json', 'C:/sales/source=FR/format=json/date=2020-01-03/order-20200103.json', 'C:/sales/source=FR/format=json/date=2020-01-04/order-20200104.json']
['C:/sales/source=FR/format=json/date=2020-01-01/order-20200102.json', 'C:/sales/source=FR/format=json/date=2020-01-02/order-20200102.json', 'C:/sales/source=FR/format=json/date=2020-01-03/order-20200103.json', 'C:/sales/source=FR/format=json/date=2020-01-04/order-20200104.json', 'C:/sa

09:55:59 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
09:56:00 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905078377,
"os.name" : Windows

09:56:00 - INFO - query: [PUT 'file://C:/sales/source=FR/format=json/date=2020-01-01/order-20200102.json' ...]
09:56:00 - INFO - query execution done
09:56:00 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
09:56:00 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
09:56:01 - INFO

09:56:12 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905078401,
"os.name" : Windows

09:56:12 - INFO - query: [PUT 'file://C:/sales/source=FR/format=json/date=2020-01-12/order-20200112.json' ...]
09:56:12 - INFO - query execution done
09:56:12 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
09:56:12 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
09:56:13 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905078405,
"os.name" : Windows

09:56:13 - INFO - query: [PUT 'file://C:/sales/source=FR/format=json/date=20

09:56:24 - INFO - query: [PUT 'file://C:/sales/source=FR/format=json/date=2020-01-23/order-20200123.json' ...]
09:56:24 - INFO - query execution done
09:56:25 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
09:56:25 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
09:56:25 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905074365,
"os.name" : Windows

09:56:25 - INFO - query: [PUT 'file://C:/sales/source=FR/format=json/date=2020-01-24/order-20200124.json' ...]
09:56:25 - INFO - query execution done
09:56:26 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-S

In [59]:
def main():
    directory_path = 'C:/sales'
    csv_file_name, csv_partition_dir, csv_local_file_path = traverse_directory(directory_path,'.parquet')
    
    stage_location = "@SALES_DWH.SOURCE.MY_INTERNAL_STG"
    
    csv_index = 0
    for file_element in csv_file_name:
        stage_path = stage_location +"/"+ csv_partition_dir[csv_index]
    
        put_result = ( 
                    get_snowpark_session().file.put( 
                        csv_local_file_path[csv_index], 
                        stage_location+"/"+csv_partition_dir[csv_index],
                        auto_compress=  False, overwrite=True, parallel=10)
                    )
        csv_index+=1
if __name__ == '__main__':
    main()

C:/sales
['C:/sales/source=US/format=parquet/date=2020-01-01/order-20200101.snappy.parquet']
['C:/sales/source=US/format=parquet/date=2020-01-01/order-20200101.snappy.parquet', 'C:/sales/source=US/format=parquet/date=2020-01-02/order-20200102.snappy.parquet']
['C:/sales/source=US/format=parquet/date=2020-01-01/order-20200101.snappy.parquet', 'C:/sales/source=US/format=parquet/date=2020-01-02/order-20200102.snappy.parquet', 'C:/sales/source=US/format=parquet/date=2020-01-03/order-20200103.snappy.parquet']
['C:/sales/source=US/format=parquet/date=2020-01-01/order-20200101.snappy.parquet', 'C:/sales/source=US/format=parquet/date=2020-01-02/order-20200102.snappy.parquet', 'C:/sales/source=US/format=parquet/date=2020-01-03/order-20200103.snappy.parquet', 'C:/sales/source=US/format=parquet/date=2020-01-04/order-20200104.snappy.parquet']
['C:/sales/source=US/format=parquet/date=2020-01-01/order-20200101.snappy.parquet', 'C:/sales/source=US/format=parquet/date=2020-01-02/order-20200102.snappy.

09:57:26 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
09:57:26 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905078437,
"os.name" : Windows

09:57:26 - INFO - query: [PUT 'file://C:/sales/source=US/format=parquet/date=2020-01-01/order-20200101.sna...]
09:57:26 - INFO - query execution done
09:57:27 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
09:57:27 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
09:57:27 - INFO

09:57:38 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905078465,
"os.name" : Windows

09:57:38 - INFO - query: [PUT 'file://C:/sales/source=US/format=parquet/date=2020-01-12/order-20200112.sna...]
09:57:38 - INFO - query execution done
09:57:39 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
09:57:39 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
09:57:39 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905074401,
"os.name" : Windows

09:57:39 - INFO - query: [PUT 'file://C:/sales/source=US/format=parquet/date

09:57:50 - INFO - query: [PUT 'file://C:/sales/source=US/format=parquet/date=2020-01-23/order-20200123.sna...]
09:57:50 - INFO - query execution done
09:57:51 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
09:57:51 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
09:57:51 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905078489,
"os.name" : Windows

09:57:51 - INFO - query: [PUT 'file://C:/sales/source=US/format=parquet/date=2020-01-24/order-20200124.sna...]
09:57:51 - INFO - query execution done
09:57:52 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-S

In [26]:
# source 스키마에 접속하기
import sys
import logging

from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark.types import StructType, StringType, StructField, StringType,LongType,DecimalType,DateType,TimestampType
from snowflake.snowpark.functions import col,lit,row_number, rank
from snowflake.snowpark import Window

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
        "ACCOUNT":"kd10688.ap-northeast-2.aws",
        "USER":"snowpark_user",
        "PASSWORD":"Test@12$4",
        "ROLE":"SYSADMIN",
        "DATABASE":"SNOWFLAKE_SAMPLE_DATA",
        "SCHEMA":"source",
        "WAREHOUSE":"SNOWPAKR_ETL_WH"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   
session = get_snowpark_session()

  

01:33:01 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
01:33:01 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
01:33:02 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905082869,
"os.name" : Windows



In [42]:
def get_snowpark_session() -> Session:
    connection_parameters = {
        "ACCOUNT": "kd10688.ap-northeast-2.aws",
        "USER": "snowpark_user",
        "PASSWORD": "Test@12$4",
        "ROLE": "SYSADMIN",
        "DATABASE": "SNOWFLAKE_SAMPLE_DATA",
        "SCHEMA": "source",
        "WAREHOUSE": "SNOWPARK_ETL_WH"
    }
    # creating snowflake session object
    session = Session.builder.configs(connection_parameters).create()

    # 웨어하우스 선택
    

    return session

In [60]:
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"kd10688.ap-northeast-2.aws",
        "USER":"snowpark_user",
        "PASSWORD":"Test@12$4",
        "ROLE":"SYSADMIN",
        "DATABASE":"SALES_DWH",
        "SCHEMA":"SOURCE",
        "WAREHOUSE":"SNOWPARK_ETL_WH"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create() 

In [61]:
def main():
    session = get_snowpark_session()

    context_df = session.sql("select current_role(), current_database(), current_schema(), current_warehouse()")
    context_df.show(2)

if __name__ == '__main__':
    main()  

04:00:31 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
04:00:31 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
04:00:31 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905090961,
"os.name" : Windows

04:00:31 - INFO - query: [select current_role(), current_database(), current_schema(), current_warehouse()...]
04:00:31 - INFO - query execution done
04:00:31 - INFO - Number of results in first chunk: 0
04:00:31 - INFO - query: [SELECT  *  FROM (select current_role(), current_database(), current_schema(), cu...]
04:00:31 - INFO - query execution done
04:00:31 - INFO - Number of results in first chunk: 1
--------------

In [67]:
def ingest_in_sales(session)-> None:
    session.sql(" \
            copy into SALES_DWH.SOURCE.IN_SALES_ORDER from ( \
            select \
            SALES_DWH.SOURCE.IN_SALES_ORDER_SEQ.nextval, \
            t.$1::text as order_id, \
            t.$2::text as customer_name, \
            t.$3::text as mobile_key,\
            t.$4::number as order_quantity, \
            t.$5::number as unit_price, \
            t.$6::number as order_valaue,  \
            t.$7::text as promotion_code , \
            t.$8::number(10,2)  as final_order_amount,\
            t.$9::number(10,2) as tax_amount,\
            t.$10::date as order_dt,\
            t.$11::text as payment_status,\
            t.$12::text as shipping_status,\
            t.$13::text as payment_method,\
            t.$14::text as payment_provider,\
            t.$15::text as mobile,\
            t.$16::text as shipping_address,\
            metadata$filename as stg_file_name,\
            metadata$file_row_number as stg_row_numer,\
            metadata$file_last_modified as stg_last_modified\
            from \
            @sales_dwh.source.my_internal_stg/source=IN/format=csv/ \
            (                                                             \
                file_format => 'sales_dwh.common.my_csv_format'           \
            ) t  )  on_error = 'Continue'     \
            "
            ).collect()
def main():
    ingest_in_sales(session)
if __name__ == '__main__':
    main()
    

04:03:28 - INFO - query: [copy into SALES_DWH.SOURCE.IN_SALES_ORDER from (             select             ...]
04:03:29 - INFO - query execution done
04:03:29 - INFO - Number of results in first chunk: 31


In [57]:
session.sql("select ")

a


In [66]:
df = session.sql("select current_schema()")
df.show(1)
def ingest_in_sales(session)-> None:
    session.sql(" \
            copy into SALES_DWH.SOURCE.IN_SALES_ORDER from ( \
            select \
            SALES_DWH.SOURCE.IN_SALES_ORDER_SEQ.nextval, \
            t.$1::text as order_id, \
            t.$2::text as customer_name, \
            t.$3::text as mobile_key,\
            t.$4::number as order_quantity, \
            t.$5::number as unit_price, \
            t.$6::number as order_valaue,  \
            t.$7::text as promotion_code , \
            t.$8::number(10,2)  as final_order_amount,\
            t.$9::number(10,2) as tax_amount,\
            t.$10::date as order_dt,\
            t.$11::text as payment_status,\
            t.$12::text as shipping_status,\
            t.$13::text as payment_method,\
            t.$14::text as payment_provider,\
            t.$15::text as mobile,\
            t.$16::text as shipping_address,\
            metadata$filename as stg_file_name,\
            metadata$file_row_number as stg_row_numer,\
            metadata$file_last_modified as stg_last_modified\
            from \
            @sales_dwh.source.my_internal_stg/source=IN/format=csv/ \
            (                                                             \
                file_format => 'sales_dwh.common.my_csv_format'           \
            ) t  )  on_error = 'Continue'     \
            "
            ).collect()
    
def ingest_us_sales(session)-> None:
    session.sql(' \
            copy into sales_dwh.source.us_sales_order                \
            from                                    \
            (                                       \
                select                              \
                SALES_DWH.SOURCE.IN_SALES_ORDER_SEQ.nextval, \
                $1:"Order ID"::text as orde_id,   \
                $1:"Customer Name"::text as customer_name,\
                $1:"Mobile Model"::text as mobile_key,\
                to_number($1:"Quantity") as quantity,\
                to_number($1:"Price per Unit") as unit_price,\
                to_decimal($1:"Total Price") as total_price,\
                $1:"Promotion Code"::text as promotion_code,\
                $1:"Order Amount"::number(10,2) as order_amount,\
                to_decimal($1:"Tax") as tax,\
                $1:"Order Date"::date as order_dt,\
                $1:"Payment Status"::text as payment_status,\
                $1:"Shipping Status"::text as shipping_status,\
                $1:"Payment Method"::text as payment_method,\
                $1:"Payment Provider"::text as payment_provider,\
                $1:"Phone"::text as phone,\
                $1:"Delivery Address"::text as shipping_address,\
                metadata$filename as stg_file_name,\
                metadata$file_row_number as stg_row_numer,\
                metadata$file_last_modified as stg_last_modified\
                from                                \
                    @sales_dwh.source.my_internal_stg/source=US/format=parquet/\
                    (file_format => sales_dwh.common.my_parquet_format)\
                    ) on_error = continue \
            '
            ).collect()
def ingest_fr_sales(session)-> None:
    session.sql(' \
        copy into sales_dwh.source.fr_sales_order                                \
        from                                                    \
        (                                                       \
            select                                              \
            SALES_DWH.SOURCE.FR_SALES_ORDER_SEQ.nextval,         \
            $1:"Order ID"::text as orde_id,                   \
            $1:"Customer Name"::text as customer_name,          \
            $1:"Mobile Model"::text as mobile_key,              \
            to_number($1:"Quantity") as quantity,               \
            to_number($1:"Price per Unit") as unit_price,       \
            to_decimal($1:"Total Price") as total_price,        \
            $1:"Promotion Code"::text as promotion_code,        \
            $1:"Order Amount"::number(10,2) as order_amount,    \
            to_decimal($1:"Tax") as tax,                        \
            $1:"Order Date"::date as order_dt,                  \
            $1:"Payment Status"::text as payment_status,        \
            $1:"Shipping Status"::text as shipping_status,      \
            $1:"Payment Method"::text as payment_method,        \
            $1:"Payment Provider"::text as payment_provider,    \
            $1:"Phone"::text as phone,                          \
            $1:"Delivery Address"::text as shipping_address ,    \
            metadata$filename as stg_file_name,\
            metadata$file_row_number as stg_row_numer,\
            metadata$file_last_modified as stg_last_modified\
            from                                                \
            @sales_dwh.source.my_internal_stg/source=FR/format=json/\
            (file_format => sales_dwh.common.my_json_format)\
            ) on_error=continue\
        '
        ).collect()

def main():
    #ingest in sales data
    ingest_in_sales(session)
    ingest_us_sales(session)
    ingest_fr_sales(session)
if __name__ == '__main__':
    
    main()
    

04:02:01 - INFO - query: [select current_schema()]
04:02:01 - INFO - query execution done
04:02:01 - INFO - Number of results in first chunk: 0
04:02:01 - INFO - query: [SELECT  *  FROM (select current_schema()) LIMIT 1]
04:02:01 - INFO - query execution done
04:02:01 - INFO - Number of results in first chunk: 1
----------------------
|"CURRENT_SCHEMA()"  |
----------------------
|SOURCE              |
----------------------

04:02:01 - INFO - query: [copy into SALES_DWH.SOURCE.IN_SALES_ORDER from (             select             ...]
04:02:02 - INFO - query execution done


SnowparkSQLException: (1304): 01ae2a46-0000-3bf7-0000-0000718db631: 001757 (42601): 01ae2a46-0000-3bf7-0000-0000718db631: SQL compilation error:
Table 'SALES_DWH.SOURCE.IN_SALES_ORDER' does not exist

In [ ]:
# CURATED ZONE = 로그인
# source zone에서 데이터를 사용하지 좋게 처리한다

In [2]:
import sys
import logging

from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark.functions import col,lit,row_number, rank
from snowflake.snowpark import Window

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

def get_snowpark_session() -> Session:
    connection_parameters = {
        "ACCOUNT":"kd10688.ap-northeast-2.aws",
        "USER":"JUN",
        "PASSWORD":"Rlawnstjd2@",
        "ROLE":"SYSADMIN",
        "DATABASE":"SALES_DWH",
        "SCHEMA":"source",
        "WAREHOUSE":"SNOWPARK_ETL_WH"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   

In [4]:
#  필터 함수
def filter_dataset(df, column_name, filter_criterian) -> DataFrame:
    # Payment Status = Paid
    # Shipping = Delivered
    return_df = df.filter(col(column_name) == filter_criterian)

    return return_df

In [90]:
def main():
    session = get_snowpark_session()
    session.sql("use warehouse SNOWFLAKE_ETL_DWH")
    forex_df = session.sql("select * from sales_dwh.common.exchange_rate")
    forex_df.show(2)
if __name__ == '__main__':
    
    main()

10:09:06 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
10:09:06 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
10:09:07 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905082453,
"os.name" : Windows

10:09:07 - INFO - query: [select * from sales_dwh.common.exchange_rate]
10:09:07 - INFO - query execution done
10:09:07 - INFO - Number of results in first chunk: 0
10:09:07 - INFO - query: [SELECT  *  FROM (select * from sales_dwh.common.exchange_rate) LIMIT 2]
10:09:07 - INFO - query execution done
10:09:07 - INFO - Number of results in first chunk: 2
------------------------------------------------------------------

In [101]:
def main():
    #get the session object and get dataframe
    session = get_snowpark_session()
    sales_df = session.sql("select * from in_sales_order")
    
    paid_sales_df = filter_dataset(sales_df,'PAYMENT_STATUS','Paid')
    shipped_sales_df = filter_dataset(paid_sales_df,'SHIPPING_STATUS','Delivered')
    
    country_sales_df = shipped_sales_df.with_column('Country',lit('IN')).with_column('Region',lit('APAC'))

    # join to add forex calculation => exchange_dt로 수정함
    
    sales_with_forext_df = country_sales_df.join(forex_df,country_sales_df['order_dt']==forex_df['exchange_dt'],join_type='outer')
    paid_sales_df.show(1)
    forex_df.show(1)
    sales_with_forext_df.show(1)
if __name__ == '__main__':
    main()

10:36:33 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
10:36:33 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
10:36:33 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905082485,
"os.name" : Windows

10:36:33 - INFO - query: [select * from in_sales_order]
10:36:33 - INFO - query execution done
10:36:33 - INFO - Number of results in first chunk: 0
10:36:33 - INFO - query: [SELECT "SALES_ORDER_KEY" AS "SALES_ORDER_KEY", "ORDER_ID" AS "ORDER_ID", "CUSTOM...]
10:36:33 - INFO - query execution done
10:36:33 - INFO - Number of results in first chunk: 0
10:36:33 - INFO - query: [SELECT "EXCHANGE_DT" AS "EXCHANGE_DT", "USD

In [97]:
def main():
    #get the session object and get dataframe
    session = get_snowpark_session()
    sales_df = session.sql("select * from in_sales_order")

    # 필터를 적용 ( paid이고 delivered 인 것들)
    # select * from in_sales_order where PAYMENT_STATUS = 'Paid' and SHIPPING_STATUS = 'Delivered'
    paid_sales_df = filter_dataset(sales_df,'PAYMENT_STATUS','Paid')
    shipped_sales_df = filter_dataset(paid_sales_df,'SHIPPING_STATUS','Delivered')

    # country라는 칼럼과 region이라는 칼럼을 추가하고 그 칼럼에는 각각 IN 과 APAC을 추가한다
    # select *, 'IN' as Country, 'APAC' as Region from in_sales_order where PAYMENT_STATUS = 'Paid' and SHIPPING_STATUS = 'Delivered'
    country_sales_df = shipped_sales_df.with_column('Country',lit('IN')).with_column('Region',lit('APAC'))

    # 조인 OUTER 조인 => exchange_dt로 수정함
    forex_df = session.sql("select * from sales_dwh.common.exchange_rate")
    sales_with_forext_df = country_sales_df.join(forex_df,country_sales_df['order_dt']==forex_df['exchange_dt'],join_type='outer')
    #sales_with_forext_df.show(2)

    #
    # 같은 값만 찾는다
    unique_orders = sales_with_forext_df.with_column('order_rank',rank().over(Window.partitionBy(col("order_dt")).order_by(col('_metadata_last_modified').desc()))).filter(col("order_rank")==1).select(col('SALES_ORDER_KEY').alias('unique_sales_order_key'))
    final_sales_df = unique_orders.join(sales_with_forext_df,unique_orders['unique_sales_order_key']==sales_with_forext_df['SALES_ORDER_KEY'],join_type='inner')
    final_sales_df = final_sales_df.select(
        col('SALES_ORDER_KEY'),
        col('ORDER_ID'),
        col('ORDER_DT'),
        col('CUSTOMER_NAME'),
        col('MOBILE_KEY'),
        col('Country'),
        col('Region'),
        col('ORDER_QUANTITY'),
        lit('INR').alias('LOCAL_CURRENCY'),
        col('UNIT_PRICE').alias('LOCAL_UNIT_PRICE'),
        col('PROMOTION_CODE').alias('PROMOTION_CODE'),
        col('FINAL_ORDER_AMOUNT').alias('LOCAL_TOTAL_ORDER_AMT'),
        col('TAX_AMOUNT').alias('local_tax_amt'),
        col('USD2INR').alias("Exhchange_Rate"),
        (col('FINAL_ORDER_AMOUNT')/col('USD2INR')).alias('US_TOTAL_ORDER_AMT'),
        (col('TAX_AMOUNT')/col('USD2INR')).alias('USD_TAX_AMT'),
        col('payment_status'),
        col('shipping_status'),
        col('payment_method'),
        col('payment_provider'),
        col('mobile').alias('conctact_no'),
        col('shipping_address')
    )

    #final_sales_df.show(5)
    final_sales_df.write.save_as_table("sales_dwh.curated.in_sales_order",mode="append")
    
if __name__ == '__main__':
    main()

10:23:23 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
10:23:23 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
10:23:24 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905082473,
"os.name" : Windows

10:23:24 - INFO - query: [select * from in_sales_order]
10:23:24 - INFO - query execution done
10:23:24 - INFO - Number of results in first chunk: 0
10:23:24 - INFO - query: [select * from sales_dwh.common.exchange_rate]
10:23:24 - INFO - query execution done
10:23:24 - INFO - Number of results in first chunk: 0
10:23:24 - INFO - query: [SELECT "SALES_ORDER_KEY" AS "SALES_ORDER_KEY", "ORDER_ID" AS "ORDER_ID", "CUSTOM..

In [6]:
# 미국 데이터 넣기
def main():

    #세션 접속
    session = get_snowpark_session()
    sales_df = session.sql("select * from us_sales_order")

    
    # PAYMENT_STATUS = 'Paid' and SHIPPING_STATUS = 'Delivered' 인것만 검색
    paid_sales_df = filter_dataset(sales_df,'PAYMENT_STATUS','Paid')
    shipped_sales_df = filter_dataset(paid_sales_df,'SHIPPING_STATUS','Delivered')

    # country = 'US' and Region = 'NA' 인 것만 칼럼 추가
    country_sales_df = shipped_sales_df.with_column('Country',lit('US')).with_column('Region',lit('NA'))

    # 조인 OUTER 조인
    forex_df = session.sql("select * from sales_dwh.common.exchange_rate")
    sales_with_forext_df = country_sales_df.join(forex_df,country_sales_df['order_dt']==forex_df['exchange_dt'],join_type='outer')
    #sales_with_forext_df.show(2)

    # 같은 값 찾기
    print(sales_with_forext_df.count())
    unique_orders = sales_with_forext_df.with_column('order_rank',rank().over(Window.partitionBy(col("order_dt")).order_by(col('_metadata_last_modified').desc()))).filter(col("order_rank")==1).select(col('SALES_ORDER_KEY').alias('unique_sales_order_key'))
    final_sales_df = unique_orders.join(sales_with_forext_df,unique_orders['unique_sales_order_key']==sales_with_forext_df['SALES_ORDER_KEY'],join_type='inner')
    final_sales_df = final_sales_df.select(
        col('SALES_ORDER_KEY'),
        col('ORDER_ID'),
        col('ORDER_DT'),
        col('CUSTOMER_NAME'),
        col('MOBILE_KEY'),
         col('Country'),
        col('Region'),
        col('ORDER_QUANTITY'),
        lit('USD').alias('LOCAL_CURRENCY'),
        col('UNIT_PRICE').alias('LOCAL_UNIT_PRICE'),
        col('PROMOTION_CODE').alias('PROMOTION_CODE'),
        col('FINAL_ORDER_AMOUNT').alias('LOCAL_TOTAL_ORDER_AMT'),
        col('TAX_AMOUNT').alias('local_tax_amt'),
        col('USD2INR').alias("Exhchange_Rate"),
        (col('FINAL_ORDER_AMOUNT')/col('USD2USD')).alias('US_TOTAL_ORDER_AMT'),
        (col('TAX_AMOUNT')/col('USD2USD')).alias('USD_TAX_AMT'),
        col('payment_status'),
        col('shipping_status'),
        col('payment_method'),
        col('payment_provider'),
        col('phone').alias('conctact_no'),
        col('shipping_address')
    )

    #final_sales_df.show(5)
    final_sales_df.write.save_as_table("sales_dwh.curated.us_sales_order",mode="append")
    
if __name__ == '__main__':
    main()

03:53:43 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
03:53:43 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
03:53:43 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905075337,
"os.name" : Windows

03:53:43 - INFO - query: [select * from us_sales_order]
03:53:44 - INFO - query execution done
03:53:44 - INFO - Number of results in first chunk: 0
03:53:44 - INFO - query: [select * from sales_dwh.common.exchange_rate]
03:53:44 - INFO - query execution done
03:53:44 - INFO - Number of results in first chunk: 0
03:53:44 - INFO - query: [SELECT "SALES_ORDER_KEY" AS "SALES_ORDER_KEY", "ORDER_ID" AS "ORDER_ID", "CUSTOM..

In [12]:
# 프랑스 데이터 넣기
def main():

    #get the session object and get dataframe
    session = get_snowpark_session()
    sales_df = session.sql("select * from us_sales_order")

    # apply filter to select only paid and delivered sale orders
    # select * from us_sales_order where PAYMENT_STATUS = 'Paid' and SHIPPING_STATUS = 'Delivered'
    paid_sales_df = filter_dataset(sales_df,'PAYMENT_STATUS','Paid')
    shipped_sales_df = filter_dataset(paid_sales_df,'SHIPPING_STATUS','Delivered')

    # adding country and region to the data frame
    # select *, 'IN' as Country, 'APAC' as Region from us_sales_order where PAYMENT_STATUS = 'Paid' and SHIPPING_STATUS = 'Delivered'
    country_sales_df = shipped_sales_df.with_column('Country',lit('FR')).with_column('Region',lit('EU'))

    # join to add forex calculation
    forex_df = session.sql("select * from sales_dwh.common.exchange_rate")
    sales_with_forext_df = country_sales_df.join(forex_df,country_sales_df['order_dt']==forex_df['exchange_dt'],join_type='outer')
    #sales_with_forext_df.show(2)

    #de-duplication
    print(sales_with_forext_df.count())
    unique_orders = sales_with_forext_df.with_column('order_rank',rank().over(Window.partitionBy(col("order_dt")).order_by(col('_metadata_last_modified').desc()))).filter(col("order_rank")==1).select(col('SALES_ORDER_KEY').alias('unique_sales_order_key'))
    final_sales_df = unique_orders.join(sales_with_forext_df,unique_orders['unique_sales_order_key']==sales_with_forext_df['SALES_ORDER_KEY'],join_type='inner')
    final_sales_df = final_sales_df.select(
        col('SALES_ORDER_KEY'),
        col('ORDER_ID'),
        col('ORDER_DT'),
        col('CUSTOMER_NAME'),
        col('MOBILE_KEY'),
        col('Country'),
        col('Region'),
        col('ORDER_QUANTITY'),
        lit('EUR').alias('LOCAL_CURRENCY'),
        col('UNIT_PRICE').alias('LOCAL_UNIT_PRICE'),
        col('PROMOTION_CODE').alias('PROMOTION_CODE'),
        col('FINAL_ORDER_AMOUNT').alias('LOCAL_TOTAL_ORDER_AMT'),
        col('TAX_AMOUNT').alias('local_tax_amt'),
        col('USD2EU').alias("EXHCHANGE_RATE"),
        (col('FINAL_ORDER_AMOUNT')/col('USD2EU')).alias('US_TOTAL_ORDER_AMT'),
        (col('TAX_AMOUNT')/col('USD2EU')).alias('USD_TAX_AMT'),
        col('payment_status'),
        col('shipping_status'),
        col('payment_method'),
        col('payment_provider'),
        col('phone').alias('conctact_no'),
        col('shipping_address')
    )

    #final_sales_df.show(5)
    final_sales_df.write.save_as_table("sales_dwh.curated.fr_sales_order",mode="append")
    
if __name__ == '__main__':
    main()

04:23:28 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.8.16, Platform: Windows-10-10.0.19045-SP0
04:23:28 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
04:23:29 - INFO - Snowpark Session information: 
"version" : 1.4.0,
"python.version" : 3.8.16,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1905075357,
"os.name" : Windows

04:23:29 - INFO - query: [select * from us_sales_order]
04:23:29 - INFO - query execution done
04:23:29 - INFO - Number of results in first chunk: 0
04:23:29 - INFO - query: [select * from sales_dwh.common.exchange_rate]
04:23:29 - INFO - query execution done
04:23:29 - INFO - Number of results in first chunk: 0
04:23:29 - INFO - query: [SELECT "SALES_ORDER_KEY" AS "SALES_ORDER_KEY", "ORDER_ID" AS "ORDER_ID", "CUSTOM..

In [ ]:
# dw 모델링